In [1]:
import os
import zipfile
!pip install evaluate
# zip_file_path = "./microbat.zip"
# extract_to_folder = "./data"
# if os.path.exists(zip_file_path):
#     with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
#         zip_ref.extractall(extract_to_folder)

#         inner_zip_path = os.path.join(
#             extract_to_folder, "MACCROBAT2018.zip")
#         with zipfile.ZipFile(inner_zip_path, "r") as inner_zip_ref:
#             inner_extract_folder = os.path.join(
#                 extract_to_folder, "MACCROBAT2018")
#             os.makedirs(inner_extract_folder, exist_ok=True)
#             inner_zip_ref.extractall(inner_extract_folder)

#         print(f"Đã giải nén tất cả các file vào {inner_extract_folder}")

# else:
#     print(f"Không tìm thấy file {zip_file_path}")
import wandb
from kaggle_secrets import UserSecretsClient
import nltk
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Wandb for Medical Ner")
wandb.login(key=secret_value_0)

# load tree bank dataset
tagged_sentences = nltk.corpus.treebank.tagged_sents()
print(" Number of samples :", len(tagged_sentences))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: moonlig73 (minhdeptrai). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


 Number of samples : 3914


## Preprocessing

In [2]:
from typing import List, Dict, Tuple

class preprocessing_data():
    def __init__(self, data_folder: str, tokenizer = None):
        self.list_id = [name_file.split('.')[0] for name_file in os.listdir(
            data_folder) if name_file.endswith('.txt')] # lấy unique id từ tên file
        
        self.tokenizer = tokenizer
        self.texts : List[str] = []
        self.tags : List[Dict[str, str]] = []
        
        for id in self.list_id:
            with open(os.path.join(data_folder, id + '.txt'), 'r') as file:
                self.texts.append(file.read())
            with open(os.path.join(data_folder, id + '.ann'), 'r') as file:
                # chỉ lấy tag = T
                text_bound_ann = [tag for tag in file.read().split('\n') if tag.startswith('T')]
                text_bound_lst = []

                for tag in text_bound_ann:
                    text = tag.split('\t') # ['T1', 'Age 8 19', '28-year-old']
                    label = text[1].split(' ') # ['Age', '8', '19']
                # có trường hợp start-end kh phải là số nên cần xử lý trường hợp ngoại lệ
                    try: 
                        start_offset = int(label[1])
                        end_offset = int(label[2])
                        tag_dict = {
                            "text": text[-1],
                            "label": label[0],
                            "start": str(start_offset),
                            "end": str(end_offset)
                        }
                        text_bound_lst.append(tag_dict)
                    except ValueError:
                        # print(
                        #     f"Warning: Skipping annotation in file {id}.ann due to not number in line: {tag}")
                        continue
                    
                self.tags.append(text_bound_lst)

    def process(self) -> Tuple[List[List[str]], List[List[str]]]:
        input_texts = []
        input_labels = []
        for i in range(len(self.list_id)):
            fulltext = self.texts[i]
            tags = self.tags[i]

            label_offset = []
            continuous_label_offset = []
            for tag in tags:
                offset = list(range(int(tag["start"]), int(tag["end"]) + 1)) # tag 1: [8,9,10,...19], tag 2: [20, 21, 22,..., 38]
                label_offset.append(offset)# [[8,9,10,...19], [20, 21, 22,..., 38]]
                continuous_label_offset.extend(offset)# [8,9,10,...19, 20, 21, 22,..., 38]

            all_offset = list(range(len(fulltext))) # [0,1,2,..., len(fulltext) - 1]
            zero_offset = [offset for offset in all_offset if offset not in continuous_label_offset] # [0,1,2,...,7, 39, 40, 41,...]
            zero_offset = preprocessing_data.find_continuous_ranges(zero_offset) # [[0,1,2,...,7], [39, 40, 41,...]] gom nhóm các khoảng label liên tục lại

            self.tokens = []  
            self.labels = []  
            self._merge_offset(fulltext, tags, zero_offset, label_offset)

            # Danh sách tokens đã được xử lý cho văn bản hiện tại (ví dụ: ['Patient', 'is', '28', '-', 'year', '-', 'old', '.'])
            # Danh sách labels tương ứng với tokens (ví dụ: ['O', 'O', 'B-Age', 'I-Age', 'I-Age', 'I-Age', 'I-Age', 'O']
            input_texts.append(self.tokens)
            input_labels.append(self.labels)
            
        return input_texts, input_labels
    
    def _merge_offset(self, full_text, tags, zero_offset, label_offset):
        # zero: [[0,1,2], [6,7]] label: [[3,4,5]]
        i = j = 0
        while i < len(zero_offset) and j < len(label_offset):
            if zero_offset[i][0] < label_offset[j][0]:
                self._add_zero(full_text, zero_offset, i)
                i += 1
            else:
                self._add_label(full_text, label_offset, j, tags)
                j += 1

        while i < len(zero_offset):
            self._add_zero(full_text, zero_offset, i)
            i += 1

        while j < len(label_offset):
            self._add_label(full_text, label_offset, j, tags)
            j += 1

    def _add_zero(self, full_text, offset, index):
        # lấy index đầu-đuôi, bỏ qua khoảng giữa
        start, *_, end = offset[index] if len(offset[index]) > 1 else (
            offset[index][0], offset[index][0] + 1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            ["O"] * len(text_tokens)
        )

    def _add_label(self, full_text, offset, index, tags):
        # lấy index đầu-đuôi, bỏ qua khoảng giữa
        start, *_, end = offset[index] if len(offset[index]) > 1 else (
            offset[index][0], offset[index][0] + 1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            [f"B-{tags[index]['label']}"] +
            [f"I-{tags[index]['label']}"] * (len(text_tokens) - 1)
        )
    
    @staticmethod
    def find_continuous_ranges(continuous_list:List[int]) -> List[List[int]]: # [0,1,2,...,7, 39, 40, 41,...]
        ranges = []
        start = continuous_list[0] # 0
        current = continuous_list[0] # 0
        for i in range(1, len(continuous_list)):
            if continuous_list[i] - 1 == current:
                current = continuous_list[i]
            else:
                ranges.append(list(range(start, current + 1)))
                start = continuous_list[i]
                current = continuous_list[i]

        return ranges  # [0,1,2,...,7, 39, 40, 41,...] -> [[0,1,2,...,7], [39, 40, 41,...]]
    
    @staticmethod
    def get_label2id(input_labels: List[List[str]]) -> Dict[str, int]:
        label2id = {}
        index_count = 0  # Initialize an index counter
        for sen_labels in input_labels:
            for label in sen_labels:
                if label not in label2id:  # Check if the tag is already in the dictionary
                    # Assign the current index count if not present
                    label2id[label] = index_count
                    index_count += 1  # Increment the index count for the next new label
        return label2id

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")

dataset_folder = "/kaggle/input/maccrobat2018/MACCROBAT2018"

Maccrobat_builder = preprocessing_data(dataset_folder, tokenizer)
input_texts, input_labels = Maccrobat_builder.process()

label2id = Maccrobat_builder.get_label2id(input_labels)
label2id["<pad>"] = len(label2id)
id2label = {v: k for k, v in label2id.items()}

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## Create DataLoader

In [4]:
# Split
from sklearn.model_selection import  train_test_split
inputs_train, inputs_val, labels_train, labels_val = train_test_split(input_texts, input_labels, test_size=0.2, random_state=42)

In [5]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 512
class ner_dataset(Dataset):
    def __init__(self, input_texts, input_labels, tokenizer, label2id, max_len=MAX_LEN):
        super().__init__()
        self.tokens = input_texts
        self.labels = input_labels
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, index):
        sen_token = self.tokens[index]
        sen_label = self.labels[index]

        token_encode = self.tokenizer.convert_tokens_to_ids(sen_token)
        token_encode = self.pad_and_truncate(token_encode, pad_id=self.tokenizer.pad_token_id, max_len=self.max_len)

        sen_label = [self.label2id[label] for label in sen_label]
        label_encode = self.pad_and_truncate(
            sen_label, pad_id=self.label2id['<pad>'], max_len=self.max_len)
        
        attention_mask = [1] * self.max_len

        return {
            # as_tensor: Cố gắng tránh sao chép, có thể chia sẻ bộ nhớ
            "input_ids" : torch.as_tensor(token_encode),
            "labels": torch.as_tensor(label_encode),
            "attention_mask": torch.as_tensor(attention_mask)
        }
    @staticmethod
    def pad_and_truncate(input_token: List[int], pad_id: int, max_len: int):
        if len(input_token) < max_len:
            input_token = input_token + [pad_id] * (max_len - len(input_token))
        else:
            input_token = input_token[:max_len]

        return input_token
    

train_set = ner_dataset(inputs_train, labels_train, tokenizer, label2id)
val_set = ner_dataset(inputs_val, labels_val, tokenizer, label2id)

In [6]:
train_iter = next(iter(train_set))
train_iter['input_ids'].shape

torch.Size([512])

## 4 Modeling

In [7]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    "d4data/biomedical-ner-all",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

## 5. Metric

In [8]:
import evaluate 
import numpy as np

accuracy = evaluate.load("accuracy")

# vì mình đạt pad = len(label2id) nên mình sẽ bỏ qua label này
ignore_label = label2id["<pad>"]

def compute_metrics(eval_pred):
    print(eval_pred)
    predictions, labels = eval_pred
    # bỏ qua padding
    mask = labels != ignore_label
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

## 6. Trainer

In [9]:
from transformers import TrainingArguments, Trainer
import shutil
import os

output_dir = "/kaggle/working/out_dir"
shutil.rmtree(output_dir, ignore_errors=True)
os.makedirs(output_dir, exist_ok=True)
print(f"Đã xóa và tạo lại thư mục đầu ra: {output_dir}")

training_args = TrainingArguments(
    output_dir="/kaggle/working/out_dir",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=40,
    eval_strategy="no",
    save_strategy="no",
    load_best_model_at_end=True,
    report_to=None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Đã xóa và tạo lại thư mục đầu ra: /kaggle/working/out_dir


<ipython-input-9-f574deacd276>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250209_104000-pzrgvtkf
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run /kaggle/working/out_dir
wandb: ⭐️ View project at https://wandb.ai/minhdeptrai/huggingface
wandb: 🚀 View run at https://wandb.ai/minhdeptrai/huggingface/runs/pzrgvtkf
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=200, training_loss=1.4292535400390625, metrics={'train_runtime': 167.9959, 'train_samples_per_second': 38.096, 'train_steps_per_second': 1.191, 'total_flos': 837419610931200.0, 'train_loss': 1.4292535400390625, 'epoch': 40.0})

In [10]:
def inference(sentence, model):
    input_sentence = torch.as_tensor(
        [tokenizer.convert_tokens_to_ids(sentence.split())])
    input_sentence = input_sentence.to("cuda")

    # prediction
    outputs = model(input_sentence)
    _, preds = torch.max(outputs.logits, -1)
    preds = preds[0].cpu().numpy()
    return preds
def pair_encode_decode(sentence, preds):
    # decode
    for token, pred in zip(sentence.split(), preds):
        print(f"{token}\t{id2label[pred]}")
def merge_entity(sentence, preds, model):
    merged_list = []
    tokens = sentence.split()
    if not tokens:
        return merged_list

    current_entity_tokens = []
    current_entity_label_type = None # Lưu LOẠI entity hiện tại (ví dụ: "Frequency")

    for token, pred in zip(tokens, preds):
        decode_label = model.config.id2label[pred]
        entity_prefix = decode_label.split('-')[0] # Lấy PREFIX (B-, I-, hoặc '')
        entity_type = decode_label.split('-')[-1] if '-' in decode_label else decode_label # Lấy LOẠI entity

        if entity_prefix == "B": # Nếu là nhãn "B-" (Bắt đầu entity mới)
            if current_entity_tokens: # Nếu có entity đang dang dở, KẾT THÚC entity đó
                merged_list.append((" ".join(current_entity_tokens), current_entity_label_type))
            current_entity_tokens = [token] # BẮT ĐẦU entity mới
            current_entity_label_type = entity_type # Gán LOẠI entity hiện tại

        elif entity_prefix == "I": # Nếu là nhãn "I-" (Tiếp tục entity)
            if entity_type == current_entity_label_type: # Nếu CÙNG LOẠI ENTITY với entity hiện tại
                current_entity_tokens.append(token) # THÊM TOKEN vào entity hiện tại
    
            else: # Nếu LOẠI ENTITY KHÁC hoặc CHƯA CÓ entity BẮT ĐẦU, coi như ENTITY MỚI (hoặc bỏ qua, tùy ý)
                if current_entity_tokens: # Kết thúc entity trước nếu có
                    merged_list.append((" ".join(current_entity_tokens), current_entity_label_type))
                current_entity_tokens = [token] # Bắt đầu entity mới (coi như B- entity)
                current_entity_label_type = entity_type # Gán LOẠI entity hiện tại

        elif decode_label == 'O': # Nếu là nhãn 'O'
            if current_entity_tokens: # Nếu có entity đang dang dở, KẾT THÚC entity đó
                merged_list.append((" ".join(current_entity_tokens), current_entity_label_type))
                current_entity_tokens = [] # RESET entity hiện tại
                current_entity_label_type = None

            merged_list.append(("O", token))

    if current_entity_tokens: # Xử lý ENTITY CUỐI CÙNG nếu có
        merged_list.append((" ".join(current_entity_tokens), current_entity_label_type))

    return merged_list

In [11]:
sentence = """A 48 year - old female presented with vaginal bleeding and abnormal Pap smears .
Upon diagnosis of invasive non - keratinizing SCC of the cervix ,
she underwent a radical hysterectomy with salpingo - oophorectomy
which demonstrated positive spread to the pelvic lymph nodes and the parametrium .
Pathological examination revealed that the tumour also extensively involved the lower uterine segment .
"""


preds = inference(sentence, model)
results = pair_encode_decode(sentence, preds)
print(results)
results = merge_entity(sentence, preds, model)
print(results)

A	O
48	B-Age
year	I-Age
-	I-Age
old	I-Age
female	B-Sex
presented	O
with	O
vaginal	B-Therapeutic_procedure
bleeding	B-Detailed_description
and	O
abnormal	B-Lab_value
Pap	O
smears	O
.	O
Upon	O
diagnosis	O
of	O
invasive	B-Lab_value
non	B-Lab_value
-	O
keratinizing	I-Detailed_description
SCC	O
of	O
the	O
cervix	O
,	O
she	O
underwent	O
a	O
radical	B-Lab_value
hysterectomy	B-Medication
with	O
salpingo	B-Medication
-	O
oophorectomy	O
which	O
demonstrated	O
positive	O
spread	O
to	O
the	O
pelvic	B-Lab_value
lymph	O
nodes	O
and	O
the	O
parametrium	O
.	O
Pathological	O
examination	O
revealed	O
that	O
the	O
tumour	B-Therapeutic_procedure
also	O
extensively	O
involved	O
the	O
lower	O
uterine	O
segment	O
.	O
None
[('O', 'A'), ('48 year - old', 'Age'), ('female', 'Sex'), ('O', 'presented'), ('O', 'with'), ('vaginal', 'Therapeutic_procedure'), ('bleeding', 'Detailed_description'), ('O', 'and'), ('abnormal', 'Lab_value'), ('O', 'Pap'), ('O', 'smears'), ('O', '.'), ('O', 'Upon'), ('O', 'diagnosis'), ('O